In [15]:
import pandas as pd
import string
import openpyxl
import datetime
import random
import os

In [16]:
# Assuming 'User_database.xlsx' is in the same directory as the notebook/script
excel_file1 = '/kaggle/input/d/mugdhazadkar/user-database/User_database.xlsx'
df_user = pd.read_excel(excel_file1)

# Create key database
headers = ['key_label', 'key_value', 'user_id', 'session_id', 'key_length', 'key_domain', 'generation_datetime']
df_keys = pd.DataFrame(columns=headers)

#Create key usage database
headers = ['key_label', 'user_id', 'session_id', 'generation_datetime']
df_key_usage = pd.DataFrame(columns=headers)

In [17]:
#user_authentication function

def authenticate_user(username, password):
    user_match = df_user[(df_user['username'] == username) & (df_user['password'] == password)]
    
    if not user_match.empty:
        return 'User authenticated'
    else:
        return 'User not recognized'

# Get user input
username = input("Enter your username: ")
password = input("Enter your password: ")

# Authenticate the user
result = authenticate_user(username, password)
print(result)

Enter your username:  
Enter your password:  


User not recognized


In [18]:
# replacement for Gokul's function
def random_string(length):
    characters = string.ascii_letters + string.digits
    return ''.join(random.choice(characters) for _ in range(length))

# Function to convert bytes to hexadecimal
def bytes_to_hex(data):
    return ''.join(['{:02x}'.format(byte) for byte in data])

#key generation function
def key_generation(user_id, session_id, key_length, key_domain):
    generation_datetime = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    key_value = random_string(key_length)
    key_label = random.randint(10000, 99999)  # Generate a key label
    key_value_hex = bytes_to_hex(key_value.encode('utf-8'))
    
    #create a dictionary with the required data
    new_key =  pd.DataFrame({'key_label':[key_label], 'key_value':[key_value_hex], 'user_id':[user_id], 'session_id':[session_id], 'key_length':[key_length], 'key_domain':[key_domain], 'generation_datetime':[generation_datetime]})
   
    # Use df.append to add the new data entry to the DataFrame
    global df_keys
    df_keys = pd.concat([df_keys, new_key], ignore_index=True)
    print(f"Key generated with key label {key_label}")
    return df_keys


In [30]:
def key_sharing(key_label, user_id, session_id):
    #the session_ID here is to record the session during which the key was shared. 
    generation_datetime = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    global df_keys
    user_key_exists = df_keys[(df_keys['user_id'] == user_id) & (df_keys['key_label'] == key_label)]
    if not user_key_exists.empty:
        # Finding the row and column number
        row_number = user_key_exists.index[0]
        #row_number = df_keys.index[df_keys['key_label'] == key_label].tolist()
        column_name = "key_value"
        column_number = df_keys.columns.get_loc(column_name)
        # finding a particular key value based on row number and column number
        key_value = df_keys.iloc[row_number, column_number]
        global df_key_usage
        usage_count = pd.DataFrame({'key_label':[key_label], 'user_id':[user_id], 'session_id':[session_id], 'generation_datetime':[generation_datetime]})
        df_key_usage = pd.concat([df_key_usage, usage_count], ignore_index=True)
        print(f"The requested key is {key_value}")
        return df_key_usage
    else:
        print("Please check the user ID and key label.")

In [20]:
# Function to delete a key from the key database
def key_termination(key_label):
     global df_keys
     try:
        key_label_exists = key_label in df_keys['key_label'].values
        if key_label_exists:
        # Find the row with the specified key_label
            df_keys = df_keys.drop(df_keys[df_keys['key_label'] == key_label].index)
            return df_keys
        else:    
            print(f"Key label '{key_label}' not found in the database.")
            return df_keys
     except Exception as e:
        print(f"An error occurred: {str(e)}")

In [28]:
# Example usage of the key_generation function
user_id = "mugdha"
session_id = 101
key_length = 16
key_domain = "QURE-derived"

key_generation(user_id, session_id, key_length, key_domain)


Key generated with key label 33305


,key_label,key_value,user_id,session_id,key_length,key_domain,generation_datetime
0,38388,585965794f71304a7059526e65504859,mugdha,101,16,QURE-derived,2023-09-26 10:10:13
1,91199,44394a544243786f4659485244706265,mugdha,101,16,QURE-derived,2023-09-26 10:11:11
2,35520,583436346c6159675a4e694c43437276,mugdha,101,16,QURE-derived,2023-09-26 10:11:11
3,76651,434f6f354b36366f6637386b53315861,mugdha,101,16,QURE-derived,2023-09-26 10:11:12
4,13330,6775436b74414666423751464b493332,mugdha,101,16,QURE-derived,2023-09-26 10:11:12
5,33305,366d78534f466d3535306772576d5733,mugdha,101,16,QURE-derived,2023-09-26 10:11:13


In [22]:
delete_key = 86876
key_termination(delete_key)

Key label '86876' not found in the database.


,key_label,key_value,user_id,session_id,key_length,key_domain,generation_datetime
0,38388,585965794f71304a7059526e65504859,mugdha,101,16,QURE-derived,2023-09-26 10:10:13


In [32]:
key_label = 38388
user_id = "mugdha"
session_id = 101
key_sharing(key_label, user_id, session_id)

The requested key is 585965794f71304a7059526e65504859


,key_label,user_id,session_id,generation_datetime
0,38388,mugdha,101,2023-09-26 10:11:31
1,38388,mugdha,101,2023-09-26 10:18:27
2,38388,mugdha,101,2023-09-26 10:18:32
